# Project Management

## Internals

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import IPython.display as dis

### Param

In [69]:
_pathTasks='tasksProjet.csv'
_pathAvail='availReel.csv'
_ProjectStartDate = dt.datetime(year=2016,month=11,day=21)
_1DayTimeDelta = dt.timedelta(1)

### Load data

In [362]:
_TaskHeaders = ['TaskId','Subject','Name','WorkLoad','Who','StartDate','EndDate','Priority','PredecessorTaskId']

def LoadTask(path):
        data = pd.read_csv(path, header=0,names=_TaskHeaders, dtype={'PredecessorTaskId':object},delimiter=';',decimal=',')
        data.StartDate = pd.to_datetime(data.StartDate)
        data.EndDate = pd.to_datetime(data.EndDate)
        data.index=data.TaskId
        if len(data)!=len(data.TaskId.unique()):
            raise Exception ('Tasks Id is not unique')
        return data
def LoadAvailability(path):
    '''
    This function load an availability matrix
    '''
    availCSV = pd.read_csv(path)
    availCSV.Date = pd.to_datetime(availCSV.Date)
    availCSV.sort_values(inplace=True,by='Date')
    return availCSV
    

### Save

In [4]:
def SaveData(data,path):
    data.to_csv(path)

### Domain
* Find tasks of a personn
* Order tasks by priority then ID (just in case)
* Compute end dates
    * Find previous task of person and previous task id
    * Based on the max start date and workload, iterate to see the day when the task will be finised


In [5]:
def FindPreviousTaskOfSomeone(tasks, current):
    '''
    Find the previous task in order of doing
    Expects an order tasks pandas by priority desc and taskid 
    '''
    whosTasks = FindTasksOfSomeone(tasks,current.Who)    
    prevTasks= whosTasks.loc[:current.TaskId]
    if len(prevTasks)<2 :
        return None
    else :
        return prevTasks.iloc[-2]
    
    

In [6]:
def FindTasksOfSomeone(tasks, who):
    return tasks[tasks.Who==who]
def FindTaskById(tasks, taskid):
    return tasks.loc[int(taskid)]
def OrderTaskByPriority(tasks):
    return tasks.sort_values(by=['Priority','TaskId'], ascending=[False,True])

In [29]:
def InitEmptyOccupation(avail,startDate, endDate):
    users = avail.Who.unique()
    duration = (endDate-startDate).days
    return pd.DataFrame(data=np.zeros(shape=(duration,len(users)), dtype=np.int32),\
                        columns=users,\
                        index=np.arange(startDate,endDate, dtype='datetime64[D]'))

In [104]:
def ComputeEndDate(availability, task):
    '''
    Compute the end date based on the occupation of someone, a start date and a duration
    '''    
    availWho = availability[(availability.Who==task.Who) & (availability.Date>=task.StartDate)]  
    currentOccupation = 0
    endDate=None    
    for index,item in availWho.iterrows():
        currentOccupation=currentOccupation+item.Available
        #print 'Task %s with workload %s at Date %s & occupation %s' %(task.TaskId,task.WorkLoad, item.Date, currentOccupation)
        if(float(currentOccupation)>=float(task.WorkLoad)):
            endDate=item.Date
            break;
    if(endDate==None):
        raise Exception('Task cannot be ended',task)
    return endDate
        

In [9]:
def SetOccupiedBy(occupation,task):
    occupation.loc[task.StartDate:task.EndDate,task.Who]=task.TaskId

In [89]:
def OrderTasksByPriorityAndPredecessor(tasks):
    tasks = OrderTaskByPriority(tasks)
    indexTasks = tasks.TaskId.tolist()
    #print indexTasks
    '''
        While on the number of element
            If has predecessor
                Get its location 
                If bigger > cursor --> move after  (check if last item)          
            If moved : do not move cursor
            Else : next item
        '''
    lItems = len(indexTasks)
    cursor = 0
    moved = False 
    listOfMoved = []
    while (cursor<lItems):  
        taskI = tasks.loc[indexTasks[cursor]]
        moved=False
        #print '  Current task %s at position %s has predecessor %s'%(taskI.TaskId,cursor,taskI.PredecessorTaskId)
        if (pd.isnull(taskI.PredecessorTaskId)==False) :
            predecIdx = indexTasks.index(int(taskI.PredecessorTaskId))    
            #print '  Predecessor %s is at position %s' %(taskI.PredecessorTaskId,predecIdx)
            if(predecIdx>cursor):
                #print '  Task %s with position %s must be moved after task %s at position %s' \
                #    %(taskI.TaskId,cursor,taskI.PredecessorTaskId,predecIdx)
                #Check if this task has already been moved, if yes we have a cycle
                if(taskI.TaskId in listOfMoved):
                    raise Exception('There is a cycle between tasks found on ',taskI.TaskId)
                listOfMoved.append(taskI.TaskId)
                indexTasks.pop(cursor)
                #print '  Popped %s' %(indexTasks)
                Moved=True
                cursor -=1
                if(predecIdx==lItems-1):
                    indexTasks.append(taskI.TaskId)
                else:
                    indexTasks.insert(predecIdx,taskI.TaskId)
        if moved==False:
                cursor +=1
        #print '  %s' %(indexTasks)
    return tasks.loc[indexTasks]

In [209]:
def ComputeOccupation(avail,tasks):
    '''
    Computer the tasks start and end date based on 
    
    1. Order of computation is based on the priority and PredecessorTaskId
        First the algorithm order by priority then check the predecessors 
    and moves each row just after its predecessor if it is before
        The algorithm will however detect incorrect cases :
    When it parcours the tasks, it checks if the PredecessorTaskId task has a end date. 
            If not, sends an error. 
            If yes uses the maximum between the previous task of the person and the predecessor
    2. By personn iterate over its tasks and compute the dates 
        2.1 If first task : prevStartDate=_projectStartDate
            else prevStartDate=previous.EndDate+1 (we cheat and say one task per day!)
        2.2 Check the predecessor end date : 
            If predecessor.EndDate = NaT --> exception (cycle)
        2.3 Take the max between both
        2.4 Compute the end date, set it and the occupation matrix
    '''
    
    lastEndDates = pd.Series(data=np.empty(len(_avail.Who.unique())),index=_avail.Who.unique())
    lastEndDates[:]=_ProjectStartDate
    orderedTasks = OrderTasksByPriorityAndPredecessor(tasks)
    for currtask in orderedTasks.itertuples():
        #print 'The current task is : %s' %(currtask.Index)        
        previousEndDate=_ProjectStartDate-_1DayTimeDelta
        predecessorTaskEndDate=_ProjectStartDate-_1DayTimeDelta
        
        #When did the last task finished
        previousTasks = FindPreviousTaskOfSomeone(tasks, currtask)  
        if(type(previousTasks)=='pandas.core.series.Series'):
            previousEndDate=previousTasks.EndDate
            if pd.isnull(previousEndDate):
                raise Exception('Couldn''t find the end date of the previous task',currtask.TaskId)

        #When did the predecessor finished if there is one
        if(pd.isnull(currtask.PredecessorTaskId)==False):            
            predecessorTask = FindTaskById(orderedTasks, currtask.PredecessorTaskId)
            #print '   %s %s' %(predecessorTask.TaskId, predecessorTask.EndDate)
            predecessorTaskEndDate=predecessorTask.EndDate
            if pd.isnull(predecessorTaskEndDate):
                raise Exception('Couldn''t find the end date of the predecessor task',currtask.TaskId)
        latestDate = max(predecessorTaskEndDate,previousEndDate,lastEndDates[currtask.Who])
        
        #The task start the next day of the previous
        orderedTasks.loc[currtask.Index,'StartDate']=latestDate+_1DayTimeDelta
        foundEndDate=ComputeEndDate(avail,orderedTasks.loc[currtask.Index])
        orderedTasks.loc[currtask.Index,'EndDate'] = foundEndDate
        lastEndDates[currtask.Who]=foundEndDate
        '''print 'Dates from %s to %s '%(orderedTasks.loc[currtask.Index,'StartDate'],\
                                      orderedTasks.loc[currtask.Index,'EndDate'])'''
    return orderedTasks
        

    

In [12]:
def FillAllOccupationFromTasks(tasks):
    occupation = InitEmptyOccupation(_avail,_ProjectStartDate,_ProjectStartDate+dt.timedelta(365))
    for task in tasks.itertuples():
        SetOccupiedBy(occupation,task)
    return occupation

#### Presentation

In [354]:
def DrawGant(tasks,idHtml,GroupByPerson=True):
    '''
    Send the javascript necessary to draw the chart
    GroupByPersonn = true display the gant for by person, otherwise by subject
    '''
    if GroupByPerson:
        tableCreation = """var dataTable = new google.visualization.DataTable();

        dataTable.addColumn({ type: 'string', id: 'Subject' });
        dataTable.addColumn({ type: 'string', id: 'Name' });
        dataTable.addColumn({'type': 'string', 'role': 'tooltip', 'p': {'html': true}})
        dataTable.addColumn({ type: 'date', id: 'Start' });
        dataTable.addColumn({ type: 'date', id: 'End' });        
        """
    else: 
        tableCreation = """var dataTable = new google.visualization.DataTable();

        dataTable.addColumn({ type: 'string', id: 'Subject' });
        dataTable.addColumn({ type: 'string', id: 'Name' });
        dataTable.addColumn({'type': 'string', 'role': 'tooltip', 'p': {'html': true}})
        dataTable.addColumn({ type: 'date', id: 'Start' });
        dataTable.addColumn({ type: 'date', id: 'End' });
        """
    def PrintToolTipHtml(task):
        return '''createCustomHTMLContent('%s','%s','%s','%s','%s','%s','%s')'''%(task.Subject.replace("'", "\\'"),\
                                                                              task.Name.replace("'", "\\'"),\
                                                              task.StartDate.date(),task.EndDate.date(),task.WorkLoad,\
                                                              task.Who,task.PredecessorTaskId)
    def PrintDate(date):
        return 'new Date(%s,%s,%s)'%(date.year,date.month-1,date.day)
    table=''
    for gt in tasks.itertuples():
        if GroupByPerson:
            table = table + """['{}','{}',{},{},{}],
        """.format(gt.Who.replace("'", "\\'"),gt.Name.replace("'", "\\'"),\
                   PrintToolTipHtml(gt),\
                   #'test',
                   PrintDate(gt.StartDate),PrintDate(gt.EndDate+_1DayTimeDelta)) 
        else:
            table = table + """['{}','{}',{},{},{}],
        """.format(gt.Subject.replace("'", "\\'"),\
                   gt.Name.replace("'", "\\'"),\
                   PrintToolTipHtml(gt),\
                   #'test',
                   PrintDate(gt.StartDate),PrintDate(gt.EndDate+_1DayTimeDelta)) 
        #The end date is one day later otherwise it stops at 0:00 leaving a day gap
    #print 'google.visualization.arrayToDataTable([{}])'.format(table)
    js = """google.charts.load("current", {{packages:["timeline"]}});
        google.charts.setOnLoadCallback(drawChart);
         function drawChart() {{
            var container = document.getElementById('{}');
            var chart = new google.visualization.Timeline(container);
            {}
            dataTable.addRows([{}]);
            var options = {{
              timeline: {{ }},
              tooltip: {{isHtml: true}},
         }};

         chart.draw(dataTable, options);
         }}
         
         function createCustomHTMLContent(subject,name,start,end, workload, who, predecessor) {{
             return '<div style="padding:5px 5px 5px 5px;">'+
             '<h3>'+subject+'<h3/>'+
             '<h4>'+name+'</h4>'+             
             '<ul><li>'+who+'</li>'+
             '<li><strong>From</strong> '+start+' <strong>To</strong> '+end+'</li>'+
             '<li><strong>WorkLoad</strong> '+workload+'</li>'+
             '<li><strong>Predecessor</strong> '+predecessor+''+
             '</ul>'+
             '</div>'
              }}
        
        """.format(idHtml,tableCreation,table)
    #print js
    return dis.Javascript(js)

## Load Data

In [377]:
_tasks = LoadTask(_pathTasks)
_avail = LoadAvailability(_pathAvail)
_tasks = ComputeOccupation(_avail,_tasks)

In [147]:
%%html
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>

In [261]:
%%html

       <div id="timeline" style="height: 650px;width: 5000px"></div>
   

In [378]:
DrawGant(_tasks,'timeline',False)

<IPython.core.display.Javascript object>

In [352]:
%%html

       <div id="timeline2" style="height: 650px;width: 5000px"></div>

In [379]:
DrawGant(_tasks,'timeline2',True)

<IPython.core.display.Javascript object>

In [364]:
_tasks.head(10)

,TaskId,Subject,Name,WorkLoad,Who,StartDate,EndDate,Priority,PredecessorTaskId
TaskId,,,,,,,,,
12,12,Telnet taux technique,Dev + UAT,3.0,ACH,2016-11-22,2016-11-28,1100,NaN
4,4,Lot R�serves,RC - Injecter les donnees en DEV,1.0,PTI,2016-11-22,2016-11-22,1095,NaN
14,14,Chargements initiaux,Agresso,3.0,QPI,2016-11-22,2016-12-02,1090,NaN
31,31,BAU,Injection de la cloture,1.0,PTI,2016-11-23,2016-11-23,1087,NaN
15,15,Chargements initiaux,Vie,15.0,PTI,2016-11-24,2016-12-20,1085,NaN
5,5,Lot R�serves,RC - Developper les flux datamarts,1.0,DEL,2016-11-23,2016-11-23,1075,4
39,39,Scoring,Preparation et Deploiement en UAT,2.0,CGA,2016-11-22,2016-11-30,1070,NaN
40,40,Scoring,Chargement datamart en UAT,2.0,CGA,2016-12-01,2016-12-12,1060,39
41,41,Scoring,Creation univers scoring,1.0,EGA,2016-12-13,2016-12-13,1050,40


In [222]:
_tasks[_tasks.Who=='ACH']

,TaskId,Subject,Name,WorkLoad,Who,StartDate,EndDate,Priority,PredecessorTaskId
TaskId,,,,,,,,,
12,12,Telnet taux technique,Dev + UAT,3.0,ACH,2016-11-22,2016-11-28,1100,NaN
13,13,Sprint,Preparation de l'environnement,2.0,ACH,2016-11-29,2016-12-01,995,NaN
7,7,Sprint,Preparation deploiement en UAT,2.0,ACH,2016-12-02,2016-12-06,982,5
8,8,Sprint,Deployer en UAT et injecter les donnees,3.0,ACH,2016-12-07,2016-12-13,980,7
9,9,Sprint,Support UAT,3.0,ACH,2016-12-14,2016-12-20,970,8
58,58,CR (a analyser),Adaptation framework data quality - technique,20.0,ACH,2016-12-21,2017-02-06,780,NaN
29,29,BAU,"Operationnalisation dqi, kpi etc",10.0,ACH,2017-02-07,2017-02-28,700,11
48,48,Corrections,"Primes techniques (inventaires, commerciales.....",10.0,ACH,2017-03-01,2017-03-23,490,NaN
28,28,DQI,DQI,62.5,ACH,2017-03-24,2017-08-17,475,NaN


In [218]:
_avail.head(5)

,Who,Date,Available
0,ACH,2016-11-21,1.0
2020,SLA,2016-11-21,1.0
3636,PM,2016-11-21,0.0
1616,MWA,2016-11-21,1.0
4040,BAU,2016-11-21,1.0


In [203]:
_tasks.loc[[16,1]]

,TaskId,Subject,Name,WorkLoad,Who,StartDate,EndDate,Priority,PredecessorTaskId
TaskId,,,,,,,,,
16,16,BAU,BAU,90.0,BAU,2016-11-22,2017-04-25,900,NaN
1,1,PM,PM,55.0,PM,2016-11-22,2017-04-20,999,NaN


# Todo
* Create a new html page 
* Better tooltip (html ?)

In [183]:
help(max)

Help on built-in function max in module __builtin__:

max(...)
    max(iterable[, key=func]) -> value
    max(a, b, c, ...[, key=func]) -> value
    
    With a single iterable argument, return its largest item.
    With two or more arguments, return the largest argument.

